In [1]:
import matplotlib.pyplot as plt
import numpy as np
import imageio
from IPython import display
from sklearn.model_selection import KFold

from os import listdir,mkdir,rmdir
from os.path import join,isdir

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import time

%matplotlib inline
plt.rcParams['figure.figsize'] = [16,9]

In [2]:
class test_net(nn.Module):
    def __init__(self, c=3, img_size=32):
        super(test_net, self).__init__()
        
        self.mp = nn.ReLU()
        self.c1 = nn.Conv2d(3, out_channels=96,kernel_size=3,stride=1,padding=1)
        self.b1 = nn.BatchNorm2d(96)
        self.p1 = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.c2 = nn.Conv2d(96, out_channels=256,kernel_size=3,stride=1,padding=1)
        self.b2 = nn.BatchNorm2d(256)
        self.p2 = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.c3 = nn.Conv2d(256, out_channels=512,kernel_size=3,stride=1,padding=1)
        self.b3 = nn.BatchNorm2d(512)
        self.c4 = nn.Conv2d(512, out_channels=512,kernel_size=3,stride=1,padding=1)
        self.b4 = nn.BatchNorm2d(512)
        self.c5 = nn.Conv2d(512, out_channels=384,kernel_size=3,stride=1,padding=1)
        self.b5 = nn.BatchNorm2d(384)
        self.p5 = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        
        self.flatten_size = 384#int(384 * (float(img_size) / 2 / 2 / 2) ** 2)
        self.a5 = nn.AvgPool2d(int(32 / 2 / 2 / 2))
        
        self.f6 = nn.Linear(self.flatten_size, out_features=1024)
        self.b6 = nn.BatchNorm1d(1024)
        self.f8 = nn.Linear(1024, 10)
        
    def forward(self, x):
        l1 = self.p1(self.mp(self.b1(self.c1(x))))
        l2 = self.p2(self.mp(self.b2(self.c2(l1))))
        l3 = self.mp(self.b3(self.c3(l2)))
        l4 = self.mp(self.b4(self.c4(l3)))
        l5 = self.p5(self.mp(self.b5(self.c5(l4))))
        f = self.a5(l5).view(-1, self.flatten_size)
        #f  = l5.view(-1, self.flatten_size)
        l6 = self.mp(self.b6(self.f6(f)))
        l8 = self.mp(self.f8(l6))
        
        return l1,l2,l3,l4,l5,l6,l8

In [3]:
class test_dataset(Dataset):
    def __init__(self, filepath='/home/darvin/Data/cifar-10-train'):
        self.datapath = filepath
        
    def __len__(self):
        length = 0
        for folder in listdir(self.datapath):
            path_imgs = join(self.datapath, folder)
            length += len(listdir(path_imgs))
        return length
    
    def __getitem__(self, idx):
        lengths = []
        for folder in listdir(self.datapath):
            path_imgs = join(self.datapath, folder)
            lengths.append(len(listdir(path_imgs)))
        for ii in range(len(lengths)):
            if idx < lengths[ii]:
                break
            idx -= lengths[ii]
        path_imgs = join(self.datapath, listdir(self.datapath)[ii])
        path_img = join(path_imgs, listdir(path_imgs)[idx])
        img = np.array(imageio.imread(path_img))
        img = np.transpose(img, [2,0,1])
        lab = ii
        return {'img': img, 'lab': lab}
    
dataset = test_dataset()
dataloader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=6)

In [4]:
lr = 0.001
weight_decay=0.0001

net = test_net().cuda()
opt = optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)
loss = nn.CrossEntropyLoss()

for i_batch,sample_batch in enumerate(dataloader):
    opt.zero_grad()
    data = sample_batch['img'].cuda().float()
    labs = sample_batch['lab'].cuda().long()
    pred = net(data)
    L = loss(pred[-1], labs)
    L.backward()
    opt.step()
    print(i_batch, L)

0 tensor(2.3066, device='cuda:0', grad_fn=<NllLossBackward>)
1 tensor(2.1703, device='cuda:0', grad_fn=<NllLossBackward>)
2 tensor(2.1417, device='cuda:0', grad_fn=<NllLossBackward>)
3 tensor(2.0942, device='cuda:0', grad_fn=<NllLossBackward>)
4 tensor(1.9276, device='cuda:0', grad_fn=<NllLossBackward>)
5 tensor(1.9557, device='cuda:0', grad_fn=<NllLossBackward>)
6 tensor(1.7731, device='cuda:0', grad_fn=<NllLossBackward>)
7 tensor(1.8437, device='cuda:0', grad_fn=<NllLossBackward>)
8 tensor(1.8614, device='cuda:0', grad_fn=<NllLossBackward>)
9 tensor(1.8505, device='cuda:0', grad_fn=<NllLossBackward>)
10 tensor(1.7865, device='cuda:0', grad_fn=<NllLossBackward>)
11 tensor(1.7113, device='cuda:0', grad_fn=<NllLossBackward>)
12 tensor(1.7082, device='cuda:0', grad_fn=<NllLossBackward>)
13 tensor(1.8135, device='cuda:0', grad_fn=<NllLossBackward>)
14 tensor(1.5847, device='cuda:0', grad_fn=<NllLossBackward>)
15 tensor(1.6713, device='cuda:0', grad_fn=<NllLossBackward>)
16 tensor(1.5722, 

Process Process-2:
Process Process-6:
Process Process-5:
Process Process-3:
Traceback (most recent call last):
Process Process-1:
Traceback (most recent call last):
Process Process-4:
  File "/home/darvin/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/darvin/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/darvin/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/darvin/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/darvin/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/darvin/anaconda3/lib/python3.6/site-packag

Traceback (most recent call last):
  File "/home/darvin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-5673c4537c1b>", line 8, in <module>
    for i_batch,sample_batch in enumerate(dataloader):
  File "/home/darvin/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 330, in __next__
    idx, batch = self._get_batch()
  File "/home/darvin/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 309, in _get_batch
    return self.data_queue.get()
  File "/home/darvin/anaconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/darvin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/darvin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = sel

KeyboardInterrupt: 

KeyboardInterrupt


In [24]:
exec("def fx():\n    print('asdf')")

In [25]:
fx()

asdf


In [24]:
'''
SANDBOX
'''

'\nSANDBOX\n'

In [ ]:
import glob,pickle

batchfiles = glob.glob(path_data+'/*_batch*')

data_train = {}
data_train['data'] = []
data_train['labels'] = []
for file in batchfiles:
    if 'test' in file:
        data_batch = pickle.load(open(file,'rb'), encoding='bytes')
        data_test = {}
        data_test['data'] = data_batch[b'data']
        data_test['labels'] = data_batch[b'labels']
    else:
        data_batch = pickle.load(open(file,'rb'), encoding='bytes')
        data_train['data'].append(data_batch[b'data'])
        data_train['labels'].append(data_batch[b'labels'])
data_train['data'] = np.concatenate(data_train['data'],axis=0)
data_train['labels'] = np.concatenate(data_train['labels'],axis=0)

meta_data = pickle.load(open(path_data+'/batches.meta','rb'), encoding='bytes')
meta_data = [item.decode('utf-8') for item in meta_data[b'label_names']]

data_train['data'] = np.transpose(data_train['data'].reshape([-1, 3, 32,32]), [0,2,3,1])
data_test['data'] = np.transpose(data_test['data'].reshape([-1, 3, 32,32]), [0,2,3,1])

In [ ]:
path_save = '/home/darvin/Data/cifar-10-test'
if not isdir(path_save):
    mkdir(path_save)
    
for ii in range(data_test['data'].shape[0]):
    path_lab = join(path_save, str(data_test['labels'][ii]))
    if not isdir(path_lab):
        mkdir(path_lab)
    img = data_test['data'][ii,:,:,:]
    imageio.imwrite(join(path_lab, '{0:05d}'.format(ii)+'.png'), img)